In [1]:
# ! pip install openpyxl pandas numpy

In [2]:
import pandas as pd
import numpy as np

channels = pd.read_excel("../data/tg_channels.xlsx")
#df = pd.read_csv("../data/cleaned_news_exp.csv")[["message_id", "id_channel", "message", "date", "topic"]]
df = pd.read_parquet("../data/tg_news_full.parquet")[["message_id", "id_channel", "message", "date"]]

/home/mlcore/conda/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
def attach_channel_w(
    df: pd.DataFrame,
    channels_df: pd.DataFrame,
    news_id_col: str = "id_channel",
    chan_id_col: str = "id",
    subs_col: str = "subscribers",
    weight_col: str = "channel_w",) -> pd.DataFrame:
    ch = channels_df[[chan_id_col, subs_col]].copy()
    ch[chan_id_col] = pd.to_numeric(ch[chan_id_col], errors="coerce").astype("Int64")
    ch[subs_col] = pd.to_numeric(ch[subs_col], errors="coerce").fillna(0).astype(float)
    id2subs = dict(zip(ch[chan_id_col], ch[subs_col]))

    out = df.copy()
    out[news_id_col] = pd.to_numeric(out[news_id_col], errors="coerce").astype("Int64")

    subs = out[news_id_col].map(id2subs).fillna(0.0).to_numpy(dtype=np.float32)
    log_subs = np.log1p(subs)

    mn, mx = float(log_subs.min()), float(log_subs.max())
    if mx > mn:
        w = (log_subs - mn) / (mx - mn)
    else:
        w = np.zeros_like(log_subs, dtype=np.float32)

    out[weight_col] = w.astype(np.float32)
    return out


In [4]:
ch_map = (channels[["id", "name"]]
          .dropna()
          .assign(id=lambda x: pd.to_numeric(x["id"], errors="coerce"))
          .dropna(subset=["id"])
          .assign(id=lambda x: x["id"].astype(int))
          .set_index("id")["name"]
          .to_dict())

df = df.copy()
df["id_channel"] = pd.to_numeric(df["id_channel"], errors="coerce")
df["channel_name"] = df["id_channel"].map(ch_map).fillna(df["id_channel"].astype("Int64").astype(str))
df = attach_channel_w(df, channels)

In [5]:
import re
import numpy as np
import pandas as pd

def clean_news_text(t: str) -> str:
    t = t or ""
    t = re.sub(r"#\w+", " ", t)
    t = re.sub(r"[⚡️📈📉🇷🇺✅❗️🔥⬛ ⬜ ⚫ ⚪🔹]+", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

def ensure_datetime(df: pd.DataFrame, col: str = "date") -> pd.DataFrame:
    d = df.copy()
    d[col] = pd.to_datetime(d[col], utc=True, errors="coerce")
    d = d.dropna(subset=[col])
    d["date_day"] = d[col].dt.floor("D")
    return d

df = ensure_datetime(df, "date")
df["message_id"] = df["message_id"].astype(str)
df["message"] = df["message"].fillna("").astype(str).map(clean_news_text)

In [6]:
df.head()

,message_id,id_channel,message,date,channel_name,channel_w,date_day
0,275548,3,"Правительство Словакии обсудит меры, которые м...",2025-01-02 17:00:02+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00
1,275547,3,В ДТП с туристическим автобусом в Таиланде пос...,2025-01-02 16:40:53+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00
2,275546,3,Премьер Израиля Нетаньяху выписан из больницы ...,2025-01-02 16:20:12+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00
3,275545,3,Подозреваемый в подрыве автомобиля Tesla Cyber...,2025-01-02 15:54:29+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00
4,275543,3,Спецоперация. Обстановка и главные события на ...,2025-01-02 15:32:55+00:00,Риа Новости,1.0,2025-01-02 00:00:00+00:00


In [7]:
#! pip install rank_bm25

In [8]:
from rank_bm25 import BM25Okapi

def tokenize_ru(text: str):
    text = text.lower()
    text = re.sub(r"[^0-9a-zа-яё\s]+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text.split()

corpus_tok = [tokenize_ru(t) for t in df["message"].tolist()]
bm25 = BM25Okapi(corpus_tok)

In [9]:
#! pip install faiss-cpu faiss-gpu-cu12 faiss-gpu-cu11

In [10]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/multilingual-e5-large" 
encoder = SentenceTransformer(MODEL_NAME, device="cuda")  
texts = df["message"].tolist()
doc_inputs = ["passage: " + t for t in texts]

In [11]:
# E_docs = encoder.encode(
#     doc_inputs,
#     batch_size=64,
#     show_progress_bar=True,
#     normalize_embeddings=True,
# ).astype(np.float32)

In [12]:
# import faiss

# dim = E_docs.shape[1]
# index = faiss.IndexFlatIP(dim)      
# index.add(E_docs)

In [13]:
# # save
# from pathlib import Path
# import numpy as np
# import pandas as pd
# import faiss
# import pickle

# OUT = Path("indexes")
# OUT.mkdir(parents=True, exist_ok=True)

# #rowmap
# rowmap = df[["message_id","date","date_day","id_channel","channel_name"]].copy()
# rowmap.to_parquet(OUT / "rowmap.parquet", index=False)

# np.save(OUT / "E_docs_e5_large.npy", E_docs)

# faiss.write_index(index, str(OUT / "faiss_e5_large.index"))

# with open(OUT / "bm25_corpus_tok.pkl", "wb") as f:
#     pickle.dump(corpus_tok, f)


In [14]:
# load
from pathlib import Path
import numpy as np
import pandas as pd
import faiss
import pickle

INP = Path("indexes")

rowmap = pd.read_parquet(INP / "rowmap.parquet")

E_docs = np.load(INP / "E_docs_e5_large.npy")
index = faiss.read_index(str(INP / "faiss_e5_large.index"))

with open(INP / "bm25_corpus_tok.pkl", "rb") as f:
    corpus_tok = pickle.load(f)

from rank_bm25 import BM25Okapi
bm25 = BM25Okapi(corpus_tok)


In [15]:
assert len(rowmap) == E_docs.shape[0]
assert len(rowmap) == index.ntotal

In [16]:
import re
import hashlib
import numpy as np
import pandas as pd

try:
    import faiss
except Exception:
    faiss = None

_URL_RE = re.compile(r"https?://\S+|www\.\S+")
_HANDLE_RE = re.compile(r"@\w+")
_WS_RE = re.compile(r"\s+")

def _normalize_for_dedup(text: str, mask_numbers: bool = True) -> str:
    if not isinstance(text, str):
        return ""
    t = text.lower()
    t = _URL_RE.sub(" ", t)
    t = _HANDLE_RE.sub(" ", t)
    t = re.sub(r"[^\w\s%.,\-]+", " ", t, flags=re.UNICODE)
    if mask_numbers:
        t = re.sub(r"\d+(?:[.,]\d+)?", "<num>", t)
    t = _WS_RE.sub(" ", t).strip()
    return t

def _stable_hash(s: str) -> str:
    return hashlib.md5(s.encode("utf-8", errors="ignore")).hexdigest()

def _union_find(n: int):
    parent = np.arange(n, dtype=np.int32)
    rank = np.zeros(n, dtype=np.int8)
    def find(x: int) -> int:
        while parent[x] != x:
            parent[x] = parent[parent[x]]
            x = parent[x]
        return int(x)
    def union(a: int, b: int) -> None:
        ra, rb = find(a), find(b)
        if ra == rb:
            return
        if rank[ra] < rank[rb]:
            parent[ra] = rb
        elif rank[ra] > rank[rb]:
            parent[rb] = ra
        else:
            parent[rb] = ra
            rank[ra] += 1
    return find, union

def dedup_cluster_candidates_time(
    cand: pd.DataFrame,
    encoder,
    text_col: str = "message",
    date_col: str = "date_day",
    channel_col: str = "channel_name",
    score_col: str = "score_rrf",
    sim_threshold: float = 0.95,
    knn: int = 20,
    keep_per_cluster: int = 1,
    mask_numbers: bool = True,
    max_day_diff: int = 1,
    overwrite_channel: bool = True,
    channel_join: str = "; ",
) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:

    if cand is None or len(cand) == 0:
        return cand, pd.DataFrame(), pd.DataFrame()

    cand = cand.copy().reset_index(drop=True)

    dts = pd.to_datetime(cand[date_col], errors="coerce", utc=True).dt.normalize()
    cand["_dt"] = dts
    cand["_dt_str"] = cand["_dt"].dt.strftime("%Y-%m-%d").fillna("")

    norm = cand[text_col].fillna("").map(lambda s: _normalize_for_dedup(s, mask_numbers=mask_numbers))
    cand["_h"] = norm.map(_stable_hash)

    cand["_hk"] = cand["_h"].astype(str) + "|" + cand["_dt_str"].astype(str)

    if score_col in cand.columns:
        rep_idx = (
            cand.sort_values(score_col, ascending=False)
                .groupby("_hk", as_index=False)
                .head(1)
                .index.to_numpy()
        )
    else:
        rep_idx = cand.groupby("_hk", as_index=False).head(1).index.to_numpy()

    rep = cand.loc[rep_idx].copy().reset_index(drop=True)

    texts = rep[text_col].fillna("").tolist()
    X = encoder.encode(texts, normalize_embeddings=True, show_progress_bar=False).astype(np.float32)

    rep_dt = pd.to_datetime(rep["_dt"], errors="coerce", utc=True)
    rep_dt = rep_dt.dt.tz_convert(None).dt.normalize().to_numpy(dtype="datetime64[D]")

    m = len(rep)
    find, union = _union_find(m)

    if m > 1:
        if faiss is None:
            S = X @ X.T
            for i in range(m):
                js = np.where(S[i, i+1:] >= sim_threshold)[0] + (i + 1)
                for j in js:
                    if np.isnat(rep_dt[i]) or np.isnat(rep_dt[j]):
                        continue
                    day_diff = abs(int((rep_dt[i] - rep_dt[j]).astype("timedelta64[D]").astype(int)))
                    if day_diff <= max_day_diff:
                        union(i, int(j))
        else:
            idx = faiss.IndexFlatIP(X.shape[1])
            idx.add(X)
            D, I = idx.search(X, min(knn, m))
            for i in range(m):
                for score, j in zip(D[i], I[i]):
                    if j < 0 or j == i:
                        continue
                    if float(score) < sim_threshold:
                        continue
                    if np.isnat(rep_dt[i]) or np.isnat(rep_dt[j]):
                        continue
                    day_diff = abs(int((rep_dt[i] - rep_dt[j]).astype("timedelta64[D]").astype(int)))
                    if day_diff <= max_day_diff:
                        union(i, int(j))

    rep_cluster = np.array([find(i) for i in range(m)], dtype=np.int32)
    _, rep_cluster = np.unique(rep_cluster, return_inverse=True)
    rep["_rep_cluster"] = rep_cluster

    hk_to_cluster = dict(zip(rep["_hk"].tolist(), rep["_rep_cluster"].tolist()))
    cand["_cluster_id"] = cand["_hk"].map(hk_to_cluster)

    cand["_cluster_id"] = cand["_cluster_id"].fillna(-1).astype(np.int32)

    cluster_sizes = cand.groupby("_cluster_id").size()

    if channel_col in cand.columns:
        ch_joined = (
            cand.groupby("_cluster_id")[channel_col]
                .apply(lambda s: channel_join.join(sorted({str(x) for x in s.dropna().tolist()})))
        )
    else:
        ch_joined = pd.Series(dtype=str)

    if score_col in cand.columns:
        cand_dedup = (
            cand.sort_values(score_col, ascending=False)
                .groupby("_cluster_id", group_keys=False)
                .head(keep_per_cluster)
                .reset_index(drop=True)
        )
    else:
        cand_dedup = (
            cand.groupby("_cluster_id", group_keys=False)
                .head(keep_per_cluster)
                .reset_index(drop=True)
        )

    cand_dedup["cluster_size"] = cand_dedup["_cluster_id"].map(cluster_sizes).astype(int)

    if channel_col in cand.columns:
        cand_dedup["channel_all"] = cand_dedup["_cluster_id"].map(ch_joined).fillna("")
        cand_dedup["channel_primary"] = cand_dedup[channel_col].astype(str)
        if overwrite_channel and channel_col in cand_dedup.columns:
            cand_dedup[channel_col] = cand_dedup["channel_all"]

    cand_dedup = cand_dedup.drop(columns=["_h", "_hk", "_dt", "_dt_str"], errors="ignore")

    clusters = cand[["_cluster_id"]].copy()
    clusters["cluster_size"] = clusters["_cluster_id"].map(cluster_sizes).astype(int)

    members = cand[["_cluster_id"]].copy()
    for c in ["date_day", "date", "channel_name", "channel", "message_id", "score_rrf"]:
        if c in cand.columns:
            members[c] = cand[c]
    members["text_snip"] = cand[text_col].fillna("").map(lambda s: s[:250])

    return cand_dedup, clusters, members

In [17]:
import numpy as np
import pandas as pd

def snippet(t: str, n: int = 1000) -> str:
    return t[:n]

def _topk_indices_from_scores(scores: np.ndarray, k: int) -> np.ndarray:
    k = min(k, len(scores))
    if k <= 0:
        return np.array([], dtype=int)
    if k == len(scores):
        idx = np.argsort(-scores)
    else:
        idx = np.argpartition(-scores, k - 1)[:k]
        idx = idx[np.argsort(-scores[idx])]
    return idx.astype(int)

def dense_candidates_faiss(index, encoder, query: str, topN: int = 500):
    qv = encoder.encode(
        ["query: " + query],
        normalize_embeddings=True,
        show_progress_bar=False
    ).astype(np.float32)
    scores, idx = index.search(qv, topN)
    return idx[0].astype(int), scores[0].astype(np.float32)

def _compute_time_arrays(df: pd.DataFrame, rowpos: np.ndarray, anchor_date, date_col: str):
    ad = pd.to_datetime(anchor_date, utc=True).normalize()
    dts = pd.to_datetime(df.loc[rowpos, date_col], errors="coerce", utc=True).dt.normalize()
    age = (ad - dts).dt.days.to_numpy(dtype=np.float32)
    age = np.where(np.isfinite(age), age, 1e9).astype(np.float32)
    age = np.where(age < 0, 1e9, age).astype(np.float32)
    return dts, age

def _time_rank_from_age(age_days: np.ndarray) -> np.ndarray:
    order = np.argsort(age_days, kind="stable")
    rank = np.empty_like(order, dtype=np.int32)
    rank[order] = np.arange(1, len(order) + 1, dtype=np.int32)
    return rank

def hybrid_retrieve_rrf(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    k: int = 50,
    topN_each: int = 500,
    k_rrf: int = 60,
    w_dense: float = 1.0,
    w_bm25: float = 1.0,
    anchor_date: str | pd.Timestamp | None = None,
    date_col: str = "date_day",
    max_window_days: int | None = 365,
    w_time: float = 0.5,
    w_channel: float | None = None,
    channel_w_col: str = "channel_w",
) -> pd.DataFrame:
    if anchor_date is not None:
        ad = pd.to_datetime(anchor_date, utc=True).normalize()
        if date_col not in df.columns:
            raise KeyError(f"date_col='{date_col}' not found in df.columns")
        dts_all = pd.to_datetime(df[date_col], errors="coerce", utc=True).dt.normalize()
        allowed = (dts_all <= ad)
        if max_window_days is not None:
            age_all = (ad - dts_all).dt.days
            allowed &= (age_all >= 0) & (age_all <= int(max_window_days))
        allowed_np = allowed.to_numpy(dtype=bool)
    else:
        allowed_np = None

    d_idx, _ = dense_candidates_faiss(index, encoder, query, topN=topN_each)
    if allowed_np is not None and len(d_idx) > 0:
        d_idx = d_idx[allowed_np[d_idx]]
    dense_rank = {int(rowpos): r for r, rowpos in enumerate(d_idx, start=1)}

    if bm25 is None:
        union = d_idx.astype(int)
        if len(union) == 0:
            return df.iloc[[]].copy().reset_index(drop=True)

        rrf = w_dense / (k_rrf + np.arange(1, len(union) + 1, dtype=np.float32))

        rank_time = None
        if anchor_date is not None and w_time and len(union) > 0:
            _, age = _compute_time_arrays(df, union, anchor_date, date_col)
            rank_time = _time_rank_from_age(age)
            rrf = rrf + (w_time / (k_rrf + rank_time.astype(np.float32)))

        order = np.argsort(-rrf)
        union = union[order]
        rrf = rrf[order]
        if rank_time is not None:
            rank_time = rank_time[order]

        out = df.iloc[union].copy()
        out["_rowpos"] = union
        out["score_rrf"] = rrf
        out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
        out["rank_bm25"] = np.nan

        if anchor_date is not None:
            doc_day, age = _compute_time_arrays(df, union, anchor_date, date_col)
            out["doc_day"] = doc_day.dt.tz_localize(None)
            out["age_days"] = age
            if rank_time is not None:
                out["rank_time"] = rank_time

        if channel_w_col in out.columns:
            if w_channel is None:
                w_channel = 0.10 * float(np.std(out["score_rrf"].to_numpy(dtype=np.float32)) or 1.0)
            out["score_rrf"] = out["score_rrf"] + float(w_channel) * out[channel_w_col].astype(np.float32)
            out = out.sort_values("score_rrf", ascending=False)

        return out.head(k).reset_index(drop=True)

    bm_scores = bm25.get_scores(tokenize_fn(query)).astype(np.float32)
    if allowed_np is not None:
        bm_scores[~allowed_np] = -np.inf
    b_idx = _topk_indices_from_scores(bm_scores, topN_each)
    bm_rank = {int(rowpos): r for r, rowpos in enumerate(b_idx, start=1)}

    union = np.array(sorted(set(dense_rank) | set(bm_rank)), dtype=int)
    if len(union) == 0:
        return df.iloc[[]].copy().reset_index(drop=True)

    rrf = np.zeros(len(union), dtype=np.float32)
    for j, rowpos in enumerate(union):
        if rowpos in dense_rank:
            rrf[j] += w_dense / (k_rrf + dense_rank[rowpos])
        if rowpos in bm_rank:
            rrf[j] += w_bm25 / (k_rrf + bm_rank[rowpos])

    rank_time = None
    if anchor_date is not None and w_time and len(union) > 0:
        _, age = _compute_time_arrays(df, union, anchor_date, date_col)
        rank_time = _time_rank_from_age(age)
        rrf = rrf + (w_time / (k_rrf + rank_time.astype(np.float32)))

    order = np.argsort(-rrf)
    union = union[order]
    rrf = rrf[order]
    if rank_time is not None:
        rank_time = rank_time[order]

    out = df.iloc[union].copy()
    out["_rowpos"] = union
    out["score_rrf"] = rrf
    out["rank_dense"] = out["_rowpos"].map(lambda rp: dense_rank.get(int(rp), np.nan))
    out["rank_bm25"] = out["_rowpos"].map(lambda rp: bm_rank.get(int(rp), np.nan))

    if anchor_date is not None:
        doc_day, age = _compute_time_arrays(df, union, anchor_date, date_col)
        out["doc_day"] = doc_day.dt.tz_localize(None)

        out["age_days"] = age
        if rank_time is not None:
            out["rank_time"] = rank_time

    if channel_w_col in out.columns:
        if w_channel is None:
            w_channel = 0.10 * float(np.std(out["score_rrf"].to_numpy(dtype=np.float32)) or 1.0)
        out["score_rrf"] = out["score_rrf"] + float(w_channel) * out[channel_w_col].astype(np.float32)
        out = out.sort_values("score_rrf", ascending=False)

    return out.head(k).reset_index(drop=True)

In [18]:
df["date_day"].min(), df["date_day"].max()

(Timestamp('2023-09-23 00:00:00+0000', tz='UTC'),
 Timestamp('2025-09-08 00:00:00+0000', tz='UTC'))

In [19]:
# ! pip install transformers torch sentence-transformers accelerate vllm

In [21]:
from transformers import AutoTokenizer, AutoConfig
from vllm import LLM
import torch

MODEL = "Qwen/Qwen2.5-32B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL, trust_remote_code=True)

cfg = AutoConfig.from_pretrained(MODEL, trust_remote_code=True)
native_ctx = getattr(cfg, "max_position_embeddings", None)
print("GPU:", torch.cuda.get_device_name(0))
print("VRAM(GB):", round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 1))
print("Model max_position_embeddings:", native_ctx)

MAX_MODEL_LEN = 19200

model = LLM(
    model=MODEL,
    dtype="bfloat16",
    max_model_len=MAX_MODEL_LEN,
    gpu_memory_utilization=0.88
)


GPU: NVIDIA A100-SXM4-80GB
VRAM(GB): 79.3
Model max_position_embeddings: 32768
INFO 01-10 05:16:14 [utils.py:253] non-default args: {'dtype': 'bfloat16', 'max_model_len': 19200, 'gpu_memory_utilization': 0.88, 'disable_log_stats': True, 'model': 'Qwen/Qwen2.5-32B-Instruct'}
INFO 01-10 05:16:15 [model.py:514] Resolved architecture: Qwen2ForCausalLM
INFO 01-10 05:16:15 [model.py:1661] Using max model len 19200
INFO 01-10 05:16:15 [scheduler.py:230] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 01-10 05:16:17 [system_utils.py:136] We must use the `spawn` multiprocessing start method. Overriding VLLM_WORKER_MULTIPROC_METHOD to 'spawn'. See https://docs.vllm.ai/en/latest/usage/troubleshooting.html#python-multiprocessing for more information. Reasons: CUDA is initialized
(EngineCore_DP0 pid=11355) INFO 01-10 05:16:24 [core.py:93] Initializing a V1 LLM engine (v0.13.0) with config: model='Qwen/Qwen2.5-32B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-32B-I

(EngineCore_DP0 pid=11355) /home/mlcore/conda/lib/python3.10/site-packages/tvm_ffi/_optional_torch_c_dlpack.py:174: UserWarning: Failed to JIT torch c dlpack extension, EnvTensorAllocator will not be enabled.
(EngineCore_DP0 pid=11355) We recommend installing via `pip install torch-c-dlpack-ext`
(EngineCore_DP0 pid=11355)   warnings.warn(


(EngineCore_DP0 pid=11355) INFO 01-10 05:16:29 [cuda.py:351] Using FLASH_ATTN attention backend out of potential backends: ('FLASH_ATTN', 'FLASHINFER', 'TRITON_ATTN', 'FLEX_ATTENTION')


Loading safetensors checkpoint shards:   0% Completed | 0/17 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:   6% Completed | 1/17 [00:00<00:09,  1.67it/s]
Loading safetensors checkpoint shards:  12% Completed | 2/17 [00:01<00:10,  1.48it/s]
Loading safetensors checkpoint shards:  18% Completed | 3/17 [00:02<00:09,  1.44it/s]
Loading safetensors checkpoint shards:  24% Completed | 4/17 [00:02<00:09,  1.40it/s]
Loading safetensors checkpoint shards:  29% Completed | 5/17 [00:03<00:08,  1.37it/s]
Loading safetensors checkpoint shards:  35% Completed | 6/17 [00:04<00:08,  1.37it/s]
Loading safetensors checkpoint shards:  41% Completed | 7/17 [00:05<00:07,  1.35it/s]
Loading safetensors checkpoint shards:  47% Completed | 8/17 [00:05<00:06,  1.33it/s]
Loading safetensors checkpoint shards:  53% Completed | 9/17 [00:06<00:06,  1.33it/s]
Loading safetensors checkpoint shards:  59% Completed | 10/17 [00:07<00:05,  1.31it/s]
Loading safetensors checkpoint shards:  65% Completed | 11/17

(EngineCore_DP0 pid=11355) INFO 01-10 05:16:43 [default_loader.py:308] Loading weights took 12.74 seconds
(EngineCore_DP0 pid=11355) INFO 01-10 05:16:44 [gpu_model_runner.py:3659] Model loading took 61.0375 GiB memory and 17.320794 seconds
(EngineCore_DP0 pid=11355) INFO 01-10 05:16:56 [backends.py:643] Using cache directory: /home/mlcore/.cache/vllm/torch_compile_cache/acdcfb9698/rank_0_0/backbone for vLLM's torch.compile
(EngineCore_DP0 pid=11355) INFO 01-10 05:16:56 [backends.py:703] Dynamo bytecode transform time: 12.13 s
(EngineCore_DP0 pid=11355) INFO 01-10 05:17:17 [backends.py:226] Directly load the compiled graph(s) for compile range (1, 8192) from the cache, took 14.028 s
(EngineCore_DP0 pid=11355) INFO 01-10 05:17:17 [monitor.py:34] torch.compile takes 26.16 s in total
(EngineCore_DP0 pid=11355) INFO 01-10 05:17:18 [gpu_worker.py:375] Available KV cache memory: 7.22 GiB
(EngineCore_DP0 pid=11355) INFO 01-10 05:17:18 [kv_cache_utils.py:1291] GPU KV cache size: 29,568 tokens
(

Capturing CUDA graphs (mixed prefill-decode, PIECEWISE): 100%|██████████| 51/51 [00:06<00:00,  7.81it/s]
Capturing CUDA graphs (decode, FULL): 100%|██████████| 35/35 [00:03<00:00, 10.30it/s]


(EngineCore_DP0 pid=11355) INFO 01-10 05:17:29 [gpu_model_runner.py:4587] Graph capturing finished in 11 secs, took 4.11 GiB
(EngineCore_DP0 pid=11355) INFO 01-10 05:17:29 [core.py:259] init engine (profile, create kv cache, warmup model) took 44.93 seconds
INFO 01-10 05:17:31 [llm.py:360] Supported tasks: ['generate']


In [22]:
SYSTEM_PROMPT = """Ты — очень внимательный новостной аналитик. Ты пишешь аккуратный дайджест по новостям в стиле корреспондента.

Вход: запрос, актуальная дата (YYYY-MM-DD) и документы вида:
[id] date=YYYY-MM-DD channel(s)=<канал1; канал2; ...>
<текст>
Все документы датированы НЕ ПОЗЖЕ актуальной даты.

ОБЩИЕ ПРАВИЛА:
1) Пиши на русском. Используй ТОЛЬКО информацию из документов.
2) Не упоминай номера документов ([id]).
3) Не добавляй дат, которых нет в документах. Не используй “сегодня/вчера/недавно” — только YYYY-MM-DD.
4) Любые числа/точные значения/пороги/цитаты/формулировки — только если они есть в документах.
5) Каналы указывай ТОЛЬКО в дайджесте (в таймлайне каналов не будет).
6) Если рядом с актуальной датой мало данных — честно укажи последнюю дату в подборке и разрыв, без домыслов “что сейчас”.
7) Нельзя вставлять “пустые” даты и строки вида “нет данных”.

КАК ПИСАТЬ ДАЙДЖЕСТ:
- Начни с 2–4 самых свежих УНИКАЛЬНЫХ дат в подборке (это “последнее время”).
- Для каждого ключевого факта укажи источник:
  “Канал(ы) (YYYY-MM-DD): …”
- Не смешивай разные типы сигналов как одно и то же: явно различай
  “официальный курс / внебиржевой курс / расчетный курс / индекс доллара / прогноз”.
- Если встречаются повторы — объединяй.
- Пиши официально и нейтрально, как в новостном дайджесте (без эмоциональных оценок и без домыслов).

СТРУКТУРА ОТВЕТА (3 блока):

### 1) Запрос и актуальная дата
* Запрос: ...
* Актуальная дата: ...

### 2) Дайджест
Связный текст (8–20 предложений, не более!), как корреспондент:
- “Последнее время”: несколько ключевых фактов по самым свежим датам + кто сообщил.
- Затем коротко “раньше/предыстория”: несколько ключевых фактов по более ранним датам + кто сообщил.
- В конце 1–2 предложения: выводы, исходя из актуальности новостей. Можно ли дать актуальную оценку или нет. Насколько тема в целом актуальная, свежая и вирусная.

### 3) Таймлайн (полный, 1 дата = 1 строка = 1 факт, без каналов)
Полный список ВСЕХ УНИКАЛЬНЫХ дат из документов, в порядке от самой старой к самой новой (ascending).
Одна строка = одна (1) дата.
Одна строка = один (1) самый релевантный запросу факт/изменение, который явно есть в документах этой даты.

Формат строки:
* YYYY-MM-DD — что сообщили / что изменилось (1 факт)

ПРАВИЛА ТАЙМЛАЙНА (критично):
- В таймлайне НЕ УКАЗЫВАЙ каналы вообще.
- Не добавляй дат, которых нет в документах!
- Не переноси факты между датами “по смыслу”: в строке даты могут быть только факты из документов этой даты!
- Никаких причин и выводов — только “что сообщили”!
- Если на одну дату приходится несколько фактов, выбери один самый важный для запроса (остальное оставь в дайджесте)!
! ЕСЛИ ДАТЫ НЕТ ВНУТРИ КОНТЕКСТА — ЕЁ НЕЛЬЗЯ УКАЗЫВАТЬ. ЕСЛИ ФАКТА НЕТ НА ЭТУ ДАТУ В ДОКУМЕНТАХ — ЕГО НЕЛЬЗЯ ПИСАТЬ. !
"""

import numpy as np
import pandas as pd

def build_rag_context(
    query: str,
    cand: pd.DataFrame,
    anchor_date: str,
    k_docs: int = 30,
    snip_chars: int = 850,
    hot_window_days: int = 30,
    hot_ratio: float = 0.8,
) -> str:
    if cand is None or len(cand) == 0:
        return (
            f"АКТУАЛЬНАЯ ДАТА ОБЗОРА: {anchor_date}\n"
            f"ВОПРОС/ЗАПРОС:\n{query}\n\n"
            f"ИСТОЧНИКИ:\n(нет документов)\n"
        )

    c = cand.copy()

    date_col = "date_day" if "date_day" in c.columns else "date"
    score_col = "score_temporal" if "score_temporal" in c.columns else "score_rrf"

    if "age_days" not in c.columns:
        ad = pd.to_datetime(anchor_date, utc=True).normalize()
        dts = pd.to_datetime(c[date_col], errors="coerce", utc=True).dt.normalize()
        c["age_days"] = (ad - dts).dt.days.astype("float32")

    age = c["age_days"].to_numpy(dtype=np.float32)
    hot_mask = (age >= 0) & (age <= float(hot_window_days))

    c = c.sort_values(score_col, ascending=False)

    n_hot = int(round(k_docs * float(hot_ratio)))
    n_hot = max(0, min(n_hot, k_docs))

    hot_part = c[hot_mask].head(n_hot)
    rest_part = c[~hot_mask].head(k_docs - len(hot_part))
    picked = pd.concat([hot_part, rest_part], axis=0)

    dd = pd.to_datetime(picked[date_col], errors="coerce", utc=True).dt.normalize()
    picked = picked.assign(_doc_day=dd).sort_values(["_doc_day", score_col], ascending=[False, False]).head(k_docs)

    blocks = []
    for i, row in enumerate(picked.itertuples(index=False), start=1):
        date_day = getattr(row, "date_day", getattr(row, "date", ""))
        if isinstance(date_day, pd.Timestamp):
            date_day = date_day.strftime("%Y-%m-%d")
        date_day = str(date_day)[:10]

        channel = getattr(row, "channel_name")
        text = getattr(row, "message", "")

        blocks.append(f"[{i}] date={date_day} channel(s)={channel}\n document=" + snippet(str(text), snip_chars))

    return (
        f"АКТУАЛЬНАЯ ДАТА ОБЗОРА: {anchor_date}\n"
        f"ВОПРОС/ЗАПРОС:\n{query}\n\n"
        f"ИСТОЧНИКИ:\n" + "\n\n".join(blocks)
    )


In [23]:
import re, json
import pandas as pd
from vllm import SamplingParams

JUDGE_SYSTEM = """Ты — строгий эксперт по информационному поиску по новостям (в т.ч. экономическим).

Твоя задача: оценить релевантность кандидатной новости запросу. Запрос может быть:
- коротким топиком (например "курс рубля к доллару"),
- или текстом другой новости (тогда запрос описывает конкретный инфоповод).

Используй ТОЛЬКО текст кандидатного документа. Ничего не додумывай.

Шкала релевантности:
2 — документ явно про то же самое: отвечает топику ИЛИ описывает тот же инфоповод/факт/событие, что и запрос.
1 — документ связан по теме/контексту, но это немного другой инфоповод, или про то же, но без прямого соответствия.
0 — нерелевантно совсем.

Правило строгости:
ставь 2 только если связь очевидна по тексту документа; если информации недостаточно — ставь 0 или 1.

Верни строго валидный JSON и ничего больше:
{"relevance": 0|1|2}
"""


def _parse_relevance(text: str) -> int:
    text = text.strip()
    m = re.search(r"\{.*\}", text, flags=re.DOTALL)
    if m:
        blob = m.group(0)
        try:
            obj = json.loads(blob)
            val = int(obj.get("relevance", 0))
            return val if val in (0, 1, 2) else 0
        except Exception:
            pass
    m2 = re.search(r"relevance\"\s*:\s*([012])", text)
    if m2:
        return int(m2.group(1))
    return 0

def judge_filter_candidates(
    cand: pd.DataFrame,
    query: str,
    judge_llm,
    judge_tokenizer,
    *,
    keep_threshold: int = 1,     
    doc_max_chars: int = 1200,
    batch_size: int = 32,
    max_out_tokens: int = 40,
) -> pd.DataFrame:
    if cand is None or len(cand) == 0:
        return cand

    text_col = "message"
    channel_col = "channel_name"
    date_col = "date_day"

    prompts = []
    for _, row in cand.iterrows():
        doc = str(row[text_col])[:doc_max_chars]
        ch = str(row[channel_col]) if channel_col else ""
        dt = str(row[date_col]) if date_col else ""

        user_msg = (
            f"ЗАПРОС:\n{query}\n\n"
            f"КАНДИДАТ:\n"
            f"channel={ch}\n"
            f"date={dt}\n"
            f"text:\n{doc}\n"
        )

        messages = [
            {"role": "system", "content": JUDGE_SYSTEM},
            {"role": "user", "content": user_msg},
        ]
        prompt = judge_tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        prompts.append(prompt)

    sampling = SamplingParams(
        temperature=0.0,
        top_p=1.0,
        max_tokens=max_out_tokens,
    )

    relevances = []
    for i in range(0, len(prompts), batch_size):
        batch_prompts = prompts[i:i+batch_size]
        outs = judge_llm.generate(batch_prompts, sampling)
        for o in outs:
            txt = o.outputs[0].text
            relevances.append(_parse_relevance(txt))

    out_df = cand.copy()
    out_df["judge_relevance"] = relevances

    filtered = out_df[out_df["judge_relevance"] >= keep_threshold].copy()
    filtered.reset_index(drop=True, inplace=True)
    return filtered


In [24]:
# import torch

# @torch.inference_mode()
# def rag_summarize(sum_model, sum_tokenizer, query: str, cand: pd.DataFrame, anchor_date, 
#                   k_docs: int = 25, snip_chars: int = 900, max_new_tokens: int = 2000) -> str:
    
#     user = build_rag_context(query, cand, anchor_date=anchor_date, k_docs=k_docs, snip_chars=snip_chars)
#     print("built context...")
    
#     messages = [
#         {"role": "system", "content": SYSTEM_PROMPT},
#         {"role": "user", "content": user},
#     ]
#     prompt = sum_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
#     enc = sum_tokenizer(prompt, return_tensors="pt", truncation=True).to(sum_model.device)

#     out_ids = sum_model.generate(
#         **enc,
#         max_new_tokens=max_new_tokens,
#         do_sample=False,
#         eos_token_id=sum_tokenizer.eos_token_id,
#         pad_token_id=sum_tokenizer.eos_token_id,
#     )
#     prompt_len = int(enc["attention_mask"][0].sum().item())
    
#     return sum_tokenizer.decode(out_ids[0][prompt_len:], skip_special_tokens=True).strip(), user


In [52]:
from vllm import SamplingParams

def rag_summarize(
    sum_model,
    sum_tokenizer,
    query: str,
    cand: pd.DataFrame,
    anchor_date,
    k_docs: int = 25,
    snip_chars: int = 900,
    max_new_tokens: int = 2000,
    hot_window_days: int = 30,
    hot_ratio: float = 0.8,
):
    user = build_rag_context(
        query=query,
        cand=cand,
        anchor_date=str(anchor_date),
        k_docs=k_docs,
        snip_chars=snip_chars,
        hot_window_days=hot_window_days,
        hot_ratio=hot_ratio,
    )
    print("built context...")

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user},
    ]

    prompt = sum_tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    sampling = SamplingParams(
        temperature=0.0,
        top_p=1.0,
        max_tokens=max_new_tokens,
    )

    result = sum_model.generate([prompt], sampling)[0]
    text = result.outputs[0].text.strip()
    return text, user


In [53]:
def run_rag_hybrid(
    df: pd.DataFrame,
    index,
    encoder,
    bm25,
    tokenize_fn,
    query: str,
    k_retrieve: int = 50,
    topN_each: int = 500,
    k_docs: int = 25,
    snip_chars: int = 1500,
    max_new_tokens: int = 2000,
    anchor_date: str = "2025-09-04",
    max_window_days: int | None = 365,
    w_time: float = 0.5,
    w_channel: float | None = None,
    hot_window_days: int = 30,
    hot_ratio: float = 0.8,
    sum_model=None,
    sum_tokenizer=None,
    judge_llm=None,
    judge_tokenizer=None,
    judge_keep_threshold: int = 1,
    judge_batch_size: int = 32,
):
    cand = hybrid_retrieve_rrf(
        df=df,
        index=index,
        encoder=encoder,
        bm25=bm25,
        tokenize_fn=tokenize_fn,
        query=query,
        k=k_retrieve,
        topN_each=topN_each,
        k_rrf=60,
        w_dense=1.0,
        w_bm25=1.0,
        anchor_date=anchor_date,
        max_window_days=max_window_days,
        w_time=w_time,
        w_channel=w_channel,
    )
    print("retrieval done...")
    cand_before = cand

    cand_clusters = None
    members = None
    if cand is not None and len(cand) > 0 and encoder is not None:
        text_col = "message"
        cand, cand_clusters, members = dedup_cluster_candidates_time(
            cand=cand,
            encoder=encoder,
            text_col=text_col,
            score_col="score_rrf",
            sim_threshold=0.95,
            knn=30,
            keep_per_cluster=1,
            mask_numbers=False,
            max_day_diff=1,
            overwrite_channel=True,
        )
    print("clustering done...")

    cand_after_dedup = cand
    if (
        judge_llm is not None
        and judge_tokenizer is not None
        and cand is not None
        and len(cand) > 0
    ):
        cand = judge_filter_candidates(
            cand=cand,
            query=query,
            judge_llm=judge_llm,
            judge_tokenizer=judge_tokenizer,
            keep_threshold=judge_keep_threshold,
            doc_max_chars=snip_chars,
            batch_size=judge_batch_size,
        )
    print("filtering done...")

    if sum_model is None or sum_tokenizer is None:
        ctx = build_rag_context(
            query,
            cand,
            anchor_date=anchor_date,
            k_docs=min(k_docs, len(cand)) if cand is not None else 0,
            snip_chars=snip_chars,
            hot_window_days=hot_window_days,
            hot_ratio=hot_ratio,
        )
        return {
            "context": ctx,
            "candidates": cand_before,
            "candidates_dedup": cand_after_dedup,
            "candidates_filtered": cand,
            "members": members,
            "clusters": cand_clusters,
            "summary": "No LLM",
        }

    summary, ctx = rag_summarize(
        sum_model,
        sum_tokenizer,
        query,
        cand,
        k_docs=min(k_docs, len(cand)) if cand is not None else 0,
        snip_chars=snip_chars,
        max_new_tokens=max_new_tokens,
        anchor_date=anchor_date,
        hot_window_days=hot_window_days,
        hot_ratio=hot_ratio,
    )
    print("summary done...")

    return {
        "context": ctx,
        "summary": summary,
        "candidates": cand_before,
        "candidates_dedup": cand_after_dedup,
        "candidates_filtered": cand,
        "members": members,
        "clusters": cand_clusters,
    }


In [54]:
q = "Актуальный курс доллара"

out = run_rag_hybrid(
    df=df,
    index=index,
    encoder=encoder,
    bm25=bm25,
    tokenize_fn=tokenize_ru,
    query=q,
    k_retrieve=150,
    topN_each=2000,
    k_docs=30,
    snip_chars=1000,
    max_new_tokens=3000,
    anchor_date="2025-09-04",
    max_window_days=365,
    w_time=0.6,
    w_channel=None,
    hot_window_days=30,
    hot_ratio=0.8,
    sum_model=model,
    sum_tokenizer=tokenizer,
    judge_llm=model,
    judge_tokenizer=tokenizer,
    judge_keep_threshold=1,
    judge_batch_size=32)


retrieval done...
clustering done...


Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Adding requests:   0%|          | 0/32 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/32 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

...

Processed prompts:   0%|          | 0/28 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

filtering done...
built context...


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

summary done...


In [55]:
from IPython.display import display, Markdown, HTML
import re

def show_summary(summary: str):
    if summary is None:
        display(HTML("<b>summary is None</b>"))
        return

    s = str(summary)
    s = s.replace("\\n", "\n") 
    s = re.sub(r"\n{3,}", "\n\n", s).strip()

    display(Markdown(s))

show_summary(out["summary"])


### 1) Запрос и актуальная дата
* Запрос: Актуальный курс доллара
* Актуальная дата: 2025-09-04

### 2) Дайджест
Последнее время, 2025-09-04, Банки, деньги, два офшора и Риа Новости сообщили, что курс рубля будет крепче, чем ожидалось, и Минэкономразвития прогнозирует доллар по 94,3 рубля в 2025 году, 100 рублей в 2026 году и 103,5 рубля в 2027 году. Также Банки, деньги, два офшора и Риа Новости отметили, что Греф ожидает ослабления курса рубля до 85-90 рублей за доллар к концу 2025 года. Официальный курс ЦБ на 2025-09-04 составил 81 рубль за доллар, что является максимумом с 1 августа.

Ранее, 2025-09-02, Блумберг и Forbes Russia сообщили, что курс доллара может достичь 100 рублей уже к январю, и что в этом году доллар в январе на пике стоил 113,7 рубля, а на минимуме в июле — 73,9 рубля. В 2025-08-31, Блумберг и Банки, деньги, два офшора сообщили, что реальный курс рубля должен составлять 24,5 за доллар, и что рубль снова оказался одной из самых недооцененных валют в мире.

Тема курса доллара остается актуальной и вирусной, с множеством противоречивых прогнозов и оценок. Однако, на основе последних данных, можно сделать вывод, что курс рубля остается крепким, но эксперты ожидают его ослабления в ближайшее время.

### 3) Таймлайн
* 2025-09-04 — Официальный курс доллара составил 81 рубль
* 2025-09-04 — Минэкономразвития прогнозирует курс доллара на 2025-2027 годы
* 2025-09-04 — Греф ожидает ослабления курса рубля до 85-90 рублей за доллар к концу 2025 года
* 2025-09-02 — Курс доллара может достичь 100 рублей уже к январю
* 2025-09-01 — Курс доллара с мая по август держится на уровне 78–80 рублей
* 2025-08-31 — Реальный курс рубля должен составлять 24,5 за доллар
* 2025-08-18 — Официальный курс доллара на 19 августа составил 80.4256 рубля
* 2025-08-14 — Официальный курс доллара на 15 августа составил 79.7653 рубля
* 2025-08-07 — Официальный курс доллара на 8 августа составил 79.3847 рубля
* 2025-07-28 — Свежий курс доллара от ЦБ оказался меньше внебиржевого на полтора рубля
* 2025-06-30 — Индекс доллара опустился ниже 97 пунктов впервые с марта 2022 года
* 2025-06-20 — Равновесным курсом является выше 100 рублей за доллар
* 2025-06-19 — Оптимальный курс в текущей ситуации в районе 100 рублей за доллар
* 2025-06-12 — Индекс доллара опустился до минимума с весны 2022 года
* 2025-05-29 — Официальный курс доллара от ЦБ опустился ниже 79 рублей
* 2025-05-21 — Официальный курс доллара на 22 мая составил 79,75 рубля
* 2025-04-21 — Официальный курс доллара опустился ниже 81 рубля
* 2025-04-14 — Центробанк понизил курс доллара более чем на 1 рубль
* 2025-03-18 — Внебиржевой курс доллара опустился ниже 81 рубля
* 2025-03-17 — Расчетный курс доллара упал ниже 83 рублей
* 2025-03-11 — Официальный курс доллара опустился до 86 рублей
* 2024-12-24 — Официальный курс доллара упал ниже 100 рублей
* 2024-12-06 — ЦБ понизил курс доллара ниже 100, евро — ниже 107 рублей
* 2024-11-27 — Курс доллара перевалил за 111 рублей
* 2024-11-14 — Курс доллара теперь стоит выше 100 рублей

In [56]:
print("context chars:", len(out["context"]))
len(SYSTEM_PROMPT)

context chars: 8266


2745

In [57]:
show_summary(out["context"])

АКТУАЛЬНАЯ ДАТА ОБЗОРА: 2025-09-04
ВОПРОС/ЗАПРОС:
Актуальный курс доллара

ИСТОЧНИКИ:
[1] date=2025-09-04 channel(s)=Банки, деньги, два офшора
 document=Курс рубля будет крепче, чем ожидалось, и «нам всем, к сожалению, придется адаптироваться» — Минэкономразвития. Ещё в апреле ведомство прогнозировало доллар по 94,3 рубля в 2025 году, в 2026-м — 100 рублей, 103,5 рубля в 2027-м. @bankrollo

[2] date=2025-09-04 channel(s)=Риа Новости
 document=Курс рубля в прогнозе Минэкономразвития будет крепче прежних оценок, всем придется к этому адаптироваться, заявил в рамках ВЭФ Решетников. Подписаться на РИА Новости

[3] date=2025-09-04 channel(s)=Банки, деньги, два офшора
 document=Доллар поднялся до 81 рубля по официальному курсу ЦБ впервые с 1 августа. @bankrollo

[4] date=2025-09-04 channel(s)=Риа Новости
 document=Греф ожидает ослабления курса рубля до 85-90 рублей за доллар к концу 2025 года. "На мой взгляд, рубль переукрепился сейчас, и вряд ли стоит ожидать укрепления рубля. Предсказывать что-либо сложно, потому что это многофакторная история. Но то, что мы видим сегодня, ослабление политики, которая сейчас происходит, приведет к ослаблению рубля", - сказал Греф в интервью Наиле Аскер-заде на телеканале "Россия 1". При этом он затруднился прогнозировать курс на среднесрочный период. "В ближайшие три года, конечно, предсказывать очень сложно, но, опять, при нормализации геополитики, пока я не вижу чего-то, что может привести к сильному укреплению рубля", - заявил банкир. Подписаться на РИА Новости

[5] date=2025-09-02 channel(s)=Блумберг
 document=Курс доллара может достичь 100 рублей уже к январю, заявил инвестиционный эксперт Александр Бахтин. @bIoombusiness

[6] date=2025-09-02 channel(s)=Forbes Russia
 document=В этом году доллар в январе на пике стоил 113,7 рубля, а на минимуме в июле — 73,9 рубля. Дошло до того, что Bloomberg назвал российскую валюту самой лучшей в 2025 году. Еще бы, рубль укрепился с начала года на 30%. С точки зрения обычного россиянина, с национальной валютой происходит что-то необычное и неясно, каков ее справедливый курс, считает генеральный директор УК «АриКапитал» Алексей Третьяков. Каким вообще должен быть справедливый курс рубля? Чтобы ответить на этот вопрос, нужно посмотреть, каким был курс в период стабильности, сделать корректировку с учетом инфляции и тех фундаментальных изменений, которые произошли за это время. В начале 2015 года курс доллара в рублях можно назвать действительно сбалансированным. В течение последующих пяти лет (вплоть до начала пандемии коронавируса в 2020 году) курс большую часть времени оставался в рамках достаточно узкого диапазона 60–70 рублей за доллар США. Скорректируем этот курс на накопленную рублевую инфляцию. Согласно официальным д

[7] date=2025-09-01 channel(s)=Forbes Russia
 document=Курс доллара с мая по август держится на комфортном для населения уровне 78–80 рублей. Ситуация чем-то похожа на 2022 год, когда после начала Россией «спецоперации»и шока на валютном рынке курс доллара с июля по декабрь находился на уровне 60 рублей из-за дисбаланса во внешней торговле — импорт обвалился, спроса на валюту не было. На протяжении последнего года эксперты говорили, что курс определяется потоками экспорта и импорта. Однако во II квартале этого года экспорт снизился, импорт подрос, но это никак не отразилось на валютном рынке. Возможная причина — больше половины расчетов по внешней торговле идет в рублях, а курс теряет связь с реальностью. Подробнее о том, почему курс доллара не меняется вопреки росту импорта и что будет с ним дальше — рассказываем на сайте 📸: Фото Getty Images

[8] date=2025-08-31 channel(s)=Блумберг
 document=Реальный курс рубля должен составлять 24,5 за доллар, считают эксперты. Российская валюта названа одной из самых недооценённых в мире. @bIoombusiness

[9] date=2025-08-31 channel(s)=Банки, деньги, два офшора
 document=Рубль снова оказался одной из самых недооцененных валют в мире. По индексу бигмака его курс должен составлять 24,5 рубля за доллар — РИА Новости. @bankollo

[10] date=2025-08-18 channel(s)=Сигналы РЦБ
 document=🏦 ЦБ установил официальные курсы валют на 19 августа: USD: 80.4256 (+0.50%) EUR: 94.0884 (+0.40%) CNY: 11.1547 (+0.42%)

[11] date=2025-08-14 channel(s)=Сигналы РЦБ
 document=🏦 ЦБ установил официальные курсы валют на 15 августа: USD: 79.7653 (+0.20%) EUR: 93.0588 (-0.35%) CNY: 11.0940 (+0.24%)

[12] date=2025-08-07 channel(s)=Сигналы РЦБ
 document=🏦 ЦБ установил официальные курсы валют на 8 августа: USD: 79.3847 (-1.00%) EUR: 92.6636 (-0.36%) CNY: 11.0170 (-0.89%)

[13] date=2025-07-28 channel(s)=Банки, деньги, два офшора
 document=Свежий курс доллара от ЦБ оказался меньше внебиржевого на полтора рубля. @bankrollo

[14] date=2025-06-30 channel(s)=Риа Новости
 document=Индекс доллара, показывающий курс к валютам шести стран - торговых партнеров США, сегодня опускался ниже 97 пунктов впервые с марта 2022-го Подписаться на РИА Новости / Все наши каналы

[15] date=2025-06-30 channel(s)=Банки, деньги, два офшора
 document=Индекс доллара рухнул ниже 97 пунктов впервые с марта 2022 года. Он показывает стоимость американской валюты к евро, иене, швейцарскому франку, британскому фунту, канадскому доллару и шведской кроне. @bankrollo

[16] date=2025-06-20 channel(s)=Риа Новости
 document=Равновесным сейчас является валютный курс выше 100 рублей за доллар, заявил глава Сбербанка Герман Греф. Он добавил, что нынешний – гораздо меньший – курс очень негативно влияет на все экспортные отрасли российской экономики и бюджет. Официальный курс доллара на пятницу составляет 78,5 рубля.

[17] date=2025-06-19 channel(s)=Банки, деньги, два офшора
 document=Оптимальный курс в текущей ситуации в районе 100 рублей за доллар, заявил первый вице-премьер Мантуров. Сегодня при крепком рубле экспорт для базовых отраслей промышленности становится малоэффективным, отметил он. @bankrollo

[18] date=2025-06-12 channel(s)=Риа Новости
 document=Индекс доллара (курс к корзине валют шести стран - торговых партнеров США) опустился до минимума с весны 2022-го. Сейчас он находится на уровне 97,8 пункта.

[19] date=2025-05-29 channel(s)=Риа Новости
 document=Официальный курс доллара от ЦБ опустился ниже 79 рублей. Его значение на четверг - 78,5 рубля.

[20] date=2025-05-21 channel(s)=Forbes Russia
 document=Банк России установил на 22 мая официальный курс доллара на уровне 79,75 рубля. Ниже 80 рублей курс доллара опустился впервые с мая 2023 года. При этом евро подорожал до 91,29 рубля, а юань почти не изменился

[21] date=2025-04-21 channel(s)=Сигналы РЦБ
 document=ЦБ установил курс доллара ниже 81 рубля впервые с лета 2023 года ЦБ РФ с 22 апреля снизил официальный курс доллара США на 37.74 коп., до 80.7597 руб.

[22] date=2025-04-21 channel(s)=Блумберг
 document=Курс доллара опустился до 79 рублей. 🔵 Bloomberg

[23] date=2025-04-14 channel(s)=Банки, деньги, два офшора; Блумберг
 document=Центробанк понизил курс доллара более чем на 1 рубль, евро — более чем на 2. @bankrollo

[24] date=2025-03-18 channel(s)=Раньше всех. Ну почти
 document=Внебиржевой курс доллара опустился ниже 81 рубля

[25] date=2025-03-17 channel(s)=Раньше всех. Ну почти
 document=Расчетный курс доллара - ориентир для внебиржевого рынка - упал ниже 83 рублей, впервые с прошлого лета, следует из данных торгов. К 17.02 мск курс доллара падает на 2,52 рубля относительно предыдущего закрытия - до 82,87 рубля, минимума с 27 августа прошлого года.

[26] date=2025-03-11 channel(s)=Банки, деньги, два офшора
 document=ЦБ опустил курс доллара до 86 рублей. Евро упал до 93,6 рубля. @bankrollo

[27] date=2024-12-24 channel(s)=Банки, деньги, два офшора
 document=Официальный курс доллара упал ниже 100. @bankrollo

[28] date=2024-12-06 channel(s)=Банки, деньги, два офшора
 document=ЦБ понизил курс доллара ниже 100, евро — ниже 107. @bankrollo

[29] date=2024-11-27 channel(s)=Блумберг
 document=Курс доллара перевалил за 111 рублей. UPD: Курс уже 114. 🔵 Bloomberg

[30] date=2024-11-14 channel(s)=Экономика 
 document=Курс доллара теперь стоит выше 100 рублей. Это следует из данных Мосбиржи. «Стоимость доллара — 100,2428 рубля», — сообщает регулятор. Такого скачка не было несколько месяцев. Сейчас Центробанк сам регулирует цену доллара по отношению к рублю. Это стало следствием санкций США. 𝔼ℂ𝕆ℕ𝕆𝕄𝕀𝕂𝔸

In [47]:
len(out["candidates"]), len(out["candidates_dedup"]), len(out["candidates_filtered"])

(150, 124, 121)